In [9]:
import z3
from lib.ts import Ts
from lib.prove import prove_inductive, prove
from lib.state_machine import smart_contract_state_machine

In [10]:

statemachine = smart_contract_state_machine('Crowdsale')
state, stateOut = statemachine.add_state("state", z3.BitVecSort(2))
deposits, depositsOut = statemachine.add_state("deposits", z3.ArraySort(z3.BitVecSort(256), z3.BitVecSort(256)))
totalDeposits, totalDepositsOut = statemachine.add_state("totalDeposits", z3.BitVecSort(256))
raised, raisedOut = statemachine.add_state("raised", z3.BitVecSort(256))


GOAL = 10000
CLOSETIME = 10000
now = z3.BitVec('now', 256)


OPEN = 0
SUCCESS = 1
REFUND = 2





In [11]:
p = z3.BitVec('p',256)


statemachine.set_init(z3.And(
    z3.ForAll(p, deposits[p]==0),
    raised == 0,
    totalDeposits == 0,
    state == OPEN,
))



value = z3.BitVec('amount',256)

statemachine.add_tr(
    tr_name = "invest",
    parameters = (value, ),
    guard = z3.And(# now <= CLOSETIME,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(stateOut == state,
                           raisedOut == raised + value,
                           depositsOut == z3.Update(deposits,p,deposits[p]+value),
                           totalDepositsOut == totalDeposits+value,
                        )
)

statemachine.add_tr(
    tr_name = "close_success",
    parameters = None,
    guard = z3.And(raised >= GOAL,
                   ),
    transfer_func = z3.And(depositsOut == deposits,
                           raisedOut == raised,
                           totalDepositsOut == totalDeposits,
                           stateOut == SUCCESS,
                        )
)

statemachine.add_tr(
    tr_name = "close_refund",
    parameters = None,
    guard = z3.And(now > CLOSETIME,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(stateOut == REFUND,
                           depositsOut == deposits,
                           raisedOut == raised,
                           totalDepositsOut == totalDeposits,
                        )
)

p = z3.BitVec('p',256)
statemachine.add_tr(
    tr_name = "claimrefund",
    parameters = (p, ),
    guard = z3.And(state == REFUND,
                   raised < GOAL,
                   ),
    transfer_func = z3.And(depositsOut == z3.Update(deposits,p,0),
                           totalDepositsOut == totalDeposits - deposits[p],
                           raisedOut == raised,
                           stateOut == state,
                        )
)


statemachine.add_tr(
    tr_name = "withdraw",
    parameters = None,
    guard = z3.And(state == SUCCESS,
                   ),
    transfer_func = z3.And(totalDepositsOut == 0,
                           depositsOut == deposits,
                           raisedOut == raised,
                           stateOut == state,
                        )
)



In [12]:
# print(statemachine.now_state)
# statemachine.transfer("invest", amount == 100)
# print(statemachine.now_state)

trace = [
    ('close_refund', now == CLOSETIME + 1),
    ('claimrefund', p == 0x114514, now == CLOSETIME + 2),
    ('invest', value == GOAL+1, now == CLOSETIME + 3),
    ('close_success', now == CLOSETIME + 4),
    ('withdraw', now == CLOSETIME + 5),
]

statemachine.simulate(trace, show_log=False)
statemachine.add_guard('invest', now <= CLOSETIME)
# statemachine.simulate(trace, show_log=True)
statemachine.simulate(trace, show_log=False)


accept
reject
